In [786]:
# Modules
import subprocess
from IPython.display import clear_output
import pandas as pd
import re
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
import ast
from scipy.stats import norm
from scipy.optimize import minimize


In [787]:
# compile zdplaskin
def run_prep(inp_path):
    try:
        process = subprocess.Popen(
            'preprocessor.exe',
            stdout = subprocess.DEVNULL,        # ignore outputs
            stderr = subprocess.DEVNULL,        # ignore errors
            stdin = subprocess.PIPE,            # recognize input
            universal_newlines=True
        )
        
        process.stdin.write(inp_path)
        process.stdin.flush()                   # send a data

        while process.poll() is None:           # check the program state, if None, program is still in the run
            process.stdin.write('.\n')
            process.stdin.flush()
    except:
        pass
    print('check the run of preprocessor')
    return process

In [788]:
# Compile exe
def compile_zdp(name):
    compile_command = [
        'gfortran', '-o', name, 'dvode_f90_m.F90', 'zdplaskin_m.F90',
        'run_plasRxn_v2.F90', 'bolsig_x86_64_g.dll'
    ]
    
    try:
        subprocess.run(compile_command)
    except:
        pass
    print('check the compiler')

In [789]:
# Run exe
def run_exe(exe_path):
    try:
        process = subprocess.Popen(
            exe_path,
            stdout = subprocess.PIPE, # Read standard outputs
            stderr = subprocess.PIPE, # Read standard errors
            universal_newlines=True,  # outputs to str variables
            bufsize = 1               # control the size of buffer
        )

        log_flag = False             # The flag for starting log after "Caculation Start!!"
        while True:
            output = process.stdout.readline()
            if not output:
                break
            if "Calculation Start" in output:
                log_flag = True

            if log_flag:
                print(f'\r{output.strip()}           ',end='',flush=True)

            if "PRESS ENTER TO EXIT" in output:
                process.kill()        # forced shutdown
                break
            if "WARNING: BOLSIG+ convergence failed" in output:
                process.kill()        # forced shutdown
                break
    except:
        pass
    return process

In [ ]:
# Error calculation
def cal_error(exp_result):
    # Read a result
    conditions = []
    with open('qt_conditions_list.txt','r') as file:
        for line in file:
            line = line.strip()
            line = line[2:]
            conditions.append(line)
        file.close()

    species = []
    with open('qt_species_list.txt','r') as file:
        for line in file:
            line = line.rstrip()
            line = line[3:]
            species.append(line)
        file.close()

    reactions = []
    reaction_list = pd.read_csv('parameter_set.csv')
    reactions = reaction_list['Reaction'].to_list()
    df_cd = pd.read_csv('qt_conditions.txt', sep=r'\s+', header=0, names=['Time [s]']+conditions)
    df_sp = pd.read_csv('qt_densities.txt', sep=r'\s+', header=0, names=['Time [s]']+species)
    df_rx = pd.read_csv('qt_rates.txt', sep=r'\s+', header=0, names=['Time [s]']+reactions)
    top_rate = df_rx.iloc[:,1:].sum().sort_values(ascending=False)

    CH4 = (df_sp['CH4'] + df_sp['CH4(V13)'] + df_sp['CH4(V24)'])
    C2H2 = (df_sp['C2H2'] + df_sp['C2H2(V13)']+ df_sp['C2H2(V2)']+ df_sp['C2H2(V5)'])
    C2H4 = (df_sp['C2H4'] + df_sp['C2H4(V1)']+ df_sp['C2H4(V2)'])
    C2H6 = (df_sp['C2H6'] + df_sp['C2H6(V13)']+ df_sp['C2H6(V24)'])
    C3H6 = (df_sp['C3H6'] + df_sp['C3H6(V)'])
    C3H8 = (df_sp['C3H8'] + df_sp['C3H8(V1)'] + df_sp['C3H8(V2)'])
    C4H10 = (df_sp['C4H9H'])
    C5H12 = (df_sp['C5H12'])
    H2 = df_sp['H2']
    C = df_sp['C']
    H = df_sp['H']
    CH = df_sp['CH']
    CH2 = df_sp['CH2']
    CH3 = df_sp['CH3']
    C2H3 = df_sp['C2H3']
    C2H5 = df_sp['C2H5']
    C3H5 = df_sp['C3H5']
    C3H7 = df_sp['C3H7']
    C4H9 = df_sp['C4H9']

    exp = exp_result
    
    sim_XCH4 = (CH4.iloc[0] - CH4.iloc[-1])/CH4.iloc[0]*100
    sim_SH2 = 0.5*H2.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC2H6 = 2*C2H6.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC2H4 = 2*C2H4.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC2H2 = 2*C2H2.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC3H8 = 3*C3H8.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC3H6 = 3*C3H6.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC4H10 = 4*C4H10.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC5H12 = 5*C5H12.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SH = 0.25*H.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SCH = CH.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SCH2 = CH2.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SCH3 = CH3.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC2H3 = 2*C2H3.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC2H5 = 2*C2H5.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC3H5 = 3*C3H5.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC3H7 = 3*C3H7.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC4H9 = 4*C4H9.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    # sim_SH2 = H2.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC2H6 = C2H6.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC2H4 = C2H4.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC2H2 = C2H2.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC3H8 = C3H8.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC3H6 = C3H6.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC4H10 = C4H10.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC5H12 = C5H12.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SH = H.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SCH = CH.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SCH2 = CH2.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SCH3 = CH3.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC2H3 = C2H3.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC2H5 = C2H5.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC3H5 = C3H5.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC3H7 = C3H7.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100
    # sim_SC4H9 = C4H9.iloc[-1] / (H2.iloc[-1] + C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C4H10.iloc[-1] + C5H12.iloc[-1])*100


    sim = []
    sim.append(sim_XCH4)
    sim.append(sim_SH2)
    sim.append(sim_SC2H6)
    sim.append(sim_SC2H4)
    sim.append(sim_SC2H2)
    sim.append(sim_SC3H8)
    sim.append(sim_SC3H6)
    sim.append(sim_SC4H10)
    sim.append(sim_SC5H12)
    sim.append(sim_SH)
    sim.append(sim_SCH)
    sim.append(sim_SCH2)
    sim.append(sim_SCH3)
    sim.append(sim_SC2H3)
    sim.append(sim_SC2H5)
    sim.append(sim_SC3H5)
    sim.append(sim_SC3H7)
    sim.append(sim_SC4H9)

    sim2 = []
    sim2.append(sim_XCH4)
    sim2.append(sim_SH2)
    sim2.append(sim_SC2H6)
    sim2.append(sim_SC2H4)
    sim2.append(sim_SC2H2)
    sim2.append(sim_SC3H8)
    sim2.append(sim_SC3H6)
    sim2.append(sim_SC4H10)
    sim2.append(sim_SC5H12)
    sim2.append(sim_SH)
    sim2.append(sim_SCH)
    sim2.append(sim_SCH2)
    sim2.append(sim_SCH3)
    sim2.append(sim_SC2H3)
    sim2.append(sim_SC2H5)
    sim2.append(sim_SC3H5)
    sim2.append(sim_SC3H7)
    sim2.append(sim_SC4H9)

    

    err =  float(np.sum((np.asarray(exp) - np.asarray(sim2))**2))
    
    return err, top_rate, sim, df_cd['Time [s]'].iloc[-1],df_sp.iloc[-1].sort_values(ascending=False)

    

In [ ]:
result_list = ['XCH4','SH2','SC2H6','SC2H4','SC2H2','SC3H8','SC3H6','SC4','SC5+','SH','SCH','SCH2','SCH3','SC2H3','SC2H5','SC3H5','SC3H7','SC4H9']
exp_result = [16.1,77.031*0.01,12.999,1.535,1.302,3.499,0.467,0.422,2.741,0,0,0,0,0,0,0,0,0]

In [792]:
inp_path = f'kinet2t.inp\n'
exe_path = 'run_plasRxn_v2.exe'
prep_process = run_prep(inp_path)
prep_process.wait()
compile_zdp(exe_path)
exe_process = run_exe(exe_path)
exe_process.wait()
error, top_rate, sim_result, total_time, total_species = cal_error(exp_result)

check the run of preprocessor
check the compiler
PRESS ENTER TO EXIT ...           (s)           gured for BOLSIG+ solver exceeds 1.79D-01           

In [793]:
sim_result2 = [round(float(i),3) for i in sim_result]

In [794]:
df_res = pd.DataFrame({
    'res': result_list,
    'exp': exp_result,
    'sim': sim_result2
    }
)
display(df_res)
print(total_time)
print(error)

,res,exp,sim
0,XCH4,16.100,16.447
1,SH2,77.031,78.229
2,SC2H6,12.999,9.691
3,SC2H4,1.535,1.079
4,SC2H2,1.302,1.210
5,SC3H8,3.499,6.211
6,SC3H6,0.467,0.272
7,SC4,0.422,3.669
8,SC5+,2.741,11.691
9,SH,0.000,0.016


2.1128
110.7527257032974


In [795]:
print(top_rate[:50])

H + H => H2                          3.580708e+21
e + CH => e + C + H                  3.002492e+21
e + CH4 => e + CH + H2 + H           2.993867e+21
e + CH4 => e + CH3 + H               6.949782e+20
e + CH4 => e + e + H + CH3^+         6.583022e+20
CH3 + CH3 => C2H6                    5.871962e+20
e + e + CH3^+ => e + CH3             5.869856e+20
e + C2H6 => e + C2H4 + H2            2.205432e+20
C2H4 + H => C2H5                     1.797126e+20
e + CH4 => e + CH4(v24)              1.707736e+20
e + CH4 => e + CH2 + H2              1.055519e+20
e + CH4 => e + CH4(v13)              1.039862e+20
C2H5 + C2H5 => C4H9H                 8.968798e+19
CH4 + CH3^+ => H2 + C2H5^+           7.321212e+19
C4H9H + CH2 => C5H12                 7.236912e+19
e + e + C2H5^+ => e + C2H5           6.925029e+19
CH3 + C2H5 => C3H8                   6.841678e+19
e + C2H4 => e + C2H2 + H2            1.534948e+19
e + CH4(v24) => e + CH + H2 + H      5.332431e+18
e + C2H5^+ => C2H3 + H + H           3.983225e+18


In [796]:
print(total_species[:50])

CH4          2.035900e+19
H2           6.340100e+18
C            2.678200e+18
C2H6         1.953000e+17
CH4(V24)     1.413000e+17
C5H12        9.474600e+16
CH4(V13)     8.643800e+16
C3H8         8.280200e+16
C4H9H        3.717000e+16
C2H2         2.447500e+16
C2H4         2.186700e+16
C3H6         3.650300e+15
CH3          3.116900e+15
H            2.565600e+15
C3H8(V1)     5.931600e+14
C2H6(V24)    5.673700e+14
C3H8(V2)     4.998900e+14
C2H6(V13)    4.795900e+14
C2H5         2.950400e+14
C2H2(V5)     3.261800e+13
C3H6(V)      2.318700e+13
C2H3         1.337000e+13
C2H2(V2)     9.356200e+12
C2H2(V13)    6.562900e+12
CH2          2.126900e+12
CH           6.740900e+09
C2H4(V1)     2.301500e+09
C2H4(V2)     6.387300e+08
E            6.157100e+05
CH3^+        5.490400e+05
C2H5^+       6.666600e+04
Time [s]     2.112800e+00
C3H7         1.760100e-02
C3H5         2.909400e-09
H^+          1.524800e-10
C4H9         4.616600e-12
C2H          4.096500e-15
CH2^+        1.319000e-15
CH4^+       